<img align="center" 
     src="https://images.squarespace-cdn.com/content/5f05b198fd381f3436f95004/1594554017779-B4XMFH0WGYLLPFEN8IYO/unifai-logo-black.png?format=1500w&content-type=image%2Fpng" 
     alt="Unifai Logo" 
     width="20%" style="padding: 10px; ">

### Champagne Coding - February 23, 2022

### What is a “notebook”?

Notebooks are an important part of the data analysis / data science workflow. 

It allows you to integrate all of the following in one place:
* Code
* Display the output
* Add visualizations
* Add narrative text
* Add mathematical equations
* ... and more :) 

All with the purpose of making work transparent, understandable, and shareable. 


### Let's get started!

We'll start by importing some modules we know we will use, like pandas!

In [ ]:
import pandas as pd

## Part 1: Data Exploration

### Load in the data set

We are going to read in this file on heatpump data.

In [ ]:
data = pd.read_csv('LabData1.csv')

Before we do any analysis or manipulations to the data, we need to know what we're looking at. Let's start with some simple exploratory analysis so we can get a good feel for our data's main characteristics.

Use ```.keys()``` to find out what headers we have in the file. 

In [ ]:
data.keys()

It's not uncommon for column headers (also called tags or sensor IDs in industry) to be a little bit cryptic. It's a good idea to check if you have any additional knowledge of what these names mean so you can do more in depth interpretation.

The interpretation is available in the file called sensor_metadata.xlsx.

#### 💡 Exercise: Read in the sensor metadata file.
👇🏼 Test in the cell below. If you can't figure it out, take a peek at [pandas documentation](https://pandas.pydata.org/docs/user_guide/index.html#user-guide)

In [ ]:
#sensor_metadata = pd.read_excel('sensor_metadata.xlsx')

Ok, now let's see how the data actually looks like. 

Use ```.head()``` to view the first few rows of the dataframe. 

In [ ]:
data.head()

With ```.shape```, we can check the dimensionality of the dataframe as a tuple: ```(number of rows, number of columns)```

In [ ]:
data.shape

How much time is that??

#### 💡 Exercise: Find when the data starts and ends.
👇🏼 Test in the cell below. Hint [here](https://www.kite.com/python/answers/how-to-find-the-max-value-of-a-pandas-dataframe-column-in-python).

Start time

End time

With ```.describe()```, we can generate descriptive statistics of the file.

In [ ]:
data.describe()

#### Data types
There are many types of data, and having inconsistent or unexpected data types can pose some problems later. 

For example, if you have a column that is time, but it is stored as "strings", it could get confusing when you try to apply date specific manipulations. Similarly, if you are storing record ids (i.e. 1, 2, 3, 4), it's a little annoying if they're stored as floats (1.0, 2.0, 3.0...)

Here is an overview of the data types that Python supports and how they can be used.

<img align="center" 
     src="https://miro.medium.com/max/3280/1*PRXWM7hwR9HHbpe-goewFQ.png" 
     alt="Pandas datatypes" 
     width="80%" style="padding: 10px; ">
     

So let's check if the data types make sense for our data. 

In [ ]:
data.dtypes

Let's go ahead and change the timestamp column datatype to datetime.

In [ ]:
data['timestamp'] = pd.to_datetime(data['timestamp'])

How does this affect the datatypes?

In [ ]:
data.dtypes

#### Check for empty rows.
It's not uncommon in industrial data sets for some data to be missing. There are lot's of reasons for this - lack of connectivity, broken sensor, different sampling rate. You can check if any data is missing in any of the columns like this.

In [ ]:
data.isnull().sum()

You'll notice that there aren't any empty rows, which is great! 

But if you did have empty rows, there are some things to consider:
* Does data need to be consistent for my analysis? 
* If you're doing manipulations to the data (i.e. Column_A + Column_B), how does missing data affect the resulting output? 

Depending on this, you have some options:
* Should I **_delete_** rows where all columns (or a subset of the columns) are missing data?
* Should I **_fill_** the empty rows? What do you fill them with? The mean? The median? 0?

#### Timeseries Index
We're working with **timeseries** data. Timeseries data is a sequence of data indexed in time. It is collected at different points in time, and typically consists of measurements collected from the same source over a time interval.The time element is important, since we are using it to track changes over time. 

To make working with timeseries data easier, we will update our index. 

If you peek above (and below here), you'll see that the current index is a range index of numbers (0, 1, 2...).

In [ ]:
print(data.index)

Let's replace that index with our ```timestamp``` column.

In [ ]:
data.set_index('timestamp', 
               drop=True, 
               inplace=True)

data.head()

### Visualization
Now that we know a little bit about our data, let's use visualization to explore further.

We'll use ```plotly express```, which is an easy-to-use, high-level interface to Plotly, which operates on a variety of types of data and produces easy-to-style figures.

In [ ]:
import plotly.express as px

Let's start by looking at the Indoor Unit Current, ```A_I_Avg```.

In [ ]:
fig = px.line(data, 
              x = data.index, 
              y="A_I_Avg")
fig.show()

#### 💡 Exercise: Plot some of the other sensors.
👇🏼 Test in the cell below. 

#### 💡 Exercise: Plot a scatter visualization with two of the sensors.
👇🏼 Test in the cell below. Hint [here](https://plotly.com/python/line-and-scatter/).

## Part 2: Calculations

### 1. Detect fan speed using ```A_I_Avg```

The fan has 5 operating modes: 
* Off
* Quiet
* Low
* Medium
* High

There is no sensor that directly captures this, but we can estimate using the indoor unit current values as a proxy. We'll create a list of bins to capture the following: 
* Off < 0.07
* 0.07 <= Quiet < 0.1
* 0.1 <= Low < 0.13
* 0.13 <= Medium < 0.2
* 0.2 <= High

In [ ]:
from numpy import inf

edges = [-inf, 0.07, 0.1, 0.13, 0.2, +inf]
labels = ["Off", "Quiet", "Low", "Medium", "High"]

In [ ]:
data['FanSpeed']= pd.cut(data.A_I_Avg, 
                         bins=edges, 
                         labels=labels)

data.FanSpeed.head()

Let's use visualization to see how that came out!

In [ ]:
fig = px.scatter(data, 
                 x = data.index, 
                 y="A_I_Avg", 
                 color = "FanSpeed")
fig.show()

In [ ]:
data.keys()

Look's about right!

Now, let's use a scatter matrix and color the pairwise plots by fanspeed to get a feel for how the rest of the data looks.

In [ ]:
fig = px.scatter_matrix(data,
                        dimensions=["A_I_Avg", "T_AI", "P_Total_Avg"],
                        color="FanSpeed")
fig.show()

### 2. Calculate the volumetric flow rate.

The volumetric flow rate is a function of the fan speed and the current. Based on tests in the labs, we know that the equations are the following:
* High: 2056x+55.831
* Medium: 4096.9x-137.66
* Low: 6879x-334.11
* Quiet: 8510.6x-355.47

Knowing that, we started putting together the function to calculate the volumetric flow rate:

In [ ]:
import numpy as np
def calculate_vfr(row):
    """
    Calculated Volumetric Flow Rate
    """
    if row['FanSpeed'] == 'High':
        vfr = (2056 * row['A_I_Avg']) + 55.831
        
    elif row['FanSpeed'] == 'Medium':
        vfr = (4096.9 * row['A_I_Avg']) - 137.66

    else:
        vfr = np.nan
        
    return convert_vfr_to_m3s(vfr)


def convert_vfr_to_m3s(vfr):
    
    return vfr*(1/60)*(1/35.3147)

#### 💡 Exercise: Update the function with the missing VFR calculations for Low and Quiet.
👆🏼 Do this in the cell above. 

Now, let's create a column for volumetric flow rate and apply the functions we created above.

In [ ]:
data['vfr_m3s'] = data.apply(calculate_vfr, axis = 1)

data['vfr_m3s'].head()

Let's see how it looks using visualization.

⚠️ You will __only__ see the scatter plot for high and medium if you didn't update the calculate vfr function above!!!! If your VFR function was updated correctly, and you run the calculation again, you should see a scatter plot that includes Low and Quiet Fan speeds.

In [ ]:
fig = px.scatter(data, 
                 x = data.index, 
                 y="vfr_m3s", 
                 color = "FanSpeed")
fig.show()

### 2. Calculate Heat Output

We're depending on a couple of constants cp (kJ/kgC) and rho (kg/m3), which we included in the cell here.

In [ ]:
constant_cp = 1.005
constant_rho = 1.2

from statistics import mean 
def calculate_heat_output(row):
    """
    T_AI: Indoor return temp
    T_A01: Indoor supply temp (1)
    T_A02: Indoor supply temp (2)
    """
    vfr_m3s = calculate_vfr(row)
    
    supply_minus_return_temp = ((mean([row['T_AO1'], row['T_AO2']]) - row['T_AI']))
    
    heat_output_watts = 1000 * vfr_m3s * constant_cp * constant_rho * supply_minus_return_temp
    
    return heat_output_watts

In [ ]:
data['heat_output'] = data.apply(calculate_heat_output, axis = 1)

data['heat_output'].head()

Again, let's see how it looks using visualization.

⚠️ You will __only__ see the scatter plot for high and medium if you didn't update the calculate vfr function above!!!! If your VFR function was updated correctly, and you run the calculation again, you should see a scatter plot that includes Low and Quiet Fan speeds.

In [ ]:
fig = px.scatter(data, 
                 x = data.index, 
                 y="heat_output", 
                 color = "FanSpeed")
fig.show()

What is the significance of the heat output?

### 3. Calculate the Coefficient of Performance
Calculate the coefficient of performance. 

In [ ]:
def calculate_cop(row):
    
    heat_output = calculate_heat_output(row)
    
    try:
        cop = heat_output/row['P_Total_Avg']
        return cop
    
    except ZeroDivisionError:
        return np.nan

In [ ]:
data['cop'] = data.apply(calculate_cop, axis = 1)

data['cop'].head()

Again, let's see how it looks using visualization.

⚠️ You will __only__ see the scatter plot for high and medium if you didn't update the calculate vfr function above!!!! If your VFR function was updated correctly, and you run the calculation again, you should see a scatter plot that includes Low and Quiet Fan speeds.

In [ ]:
fig = px.scatter(data, 
                 x = data.index, 
                 y="cop", 
                 color = "FanSpeed")
fig.show()

Why do you think there are outliers in the data? How would you filter them?

#### 💡 Exercise: Identify when the fan is ramping or in steady state.

Hint - you can try using ```pandas diff``` [functionality](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.diff.html) on A_I_Avg to see if any obvious patterns emerge.

Plot the difference as well as the A_I_Avg sensor to see what we mean :) 

In [ ]:
import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Scatter(x = data.index, 
                         y=data.A_I_Avg, name = "A_I_Avg"))

fig.add_trace(go.Scatter(x = data.index, 
                         y=data.A_I_Avg.diff(), name = "Difference"))

fig.show()

👇🏼 Filter the data into two modes: ramping & steady.

Hint - peek above on how we filtered fan speed based on value bins!!

### 4. Create a performance map
Using the coefficient of performance and the current, build a performance map.

In [ ]:
fig = px.scatter(data, 
                 x = "A_I_Avg", 
                 y="cop", 
                 color = "FanSpeed")
fig.show()

## Save your data!

In [ ]:
data.to_excel('output.xlsx')

### 🚀 What next? How can you start using Python for your data analysis? Here are some ideas for getting started.

<br>

⬇️ __DOWNLOAD PYTHON__: Anaconda is a distribution of the Python and R programming languages for scientific computing, that aims to simplify package management and deployment. Anaconda installation [here](https://www.anaconda.com/products/individual). 
<br>

🐍 __PYTHON INTRODUCTION__: If you want an exhaustive introduction to Python, make sure to check out the official [documentation](https://docs.python.org/3/) 
<br>

🤓 __PYTHON FOR DATA ANALYSIS__: If you want additional information on how Python can be used for data analysis, check out this [ebook](https://bedford-computing.co.uk/learning/wp-content/uploads/2015/10/Python-for-Data-Analysis.pdf)
<br>

🧠 __GET INVOLVED__: You can always reach out to Alexandra (alexandra@unifai.dev) with any questions! If you like learning about data science, you can also sign up to [this meetup group](https://www.meetup.com/Women-in-Data-Science-Oslo) to get invitations to more hands on workshops! 